In [1]:
!git clone https://github.com/huggingface/transformers
%cd transformers
!pip install .
!pip install -r ./examples/multiple-choice/requirements.txt
%cd ..
!mv ./transformers/examples/multiple-choice/run_swag.py ./transformers/examples/multiple-choice/run_swag.py.bak
!cp run_swag.py ./transformers/examples/multiple-choice/run_swag.py

!mkdir -pv swag
!wget https://raw.githubusercontent.com/rowanz/swagaf/master/data/train.csv -O swag/train.csv
# !wget https://raw.githubusercontent.com/rowanz/swagaf/master/data/val.csv -O swag/val.csv
# !wget https://raw.githubusercontent.com/rowanz/swagaf/master/data/test.csv -O swag/test.csv

In [2]:
DEBUG = False

In [3]:
import warnings
warnings.simplefilter('ignore')

import pandas as pd
from tqdm.notebook import tqdm

In [4]:
train = pd.read_json("raw_data/train.json")

print(train.shape)
train.head()

(6315, 3)


,ID,Content,Questions
0,1,奉和袭美抱疾杜门见寄次韵 陆龟蒙虽失春城醉上期，下帷裁遍未裁诗。因吟郢岸百亩蕙，欲采商崖三...,"[{'Question': '下列对这首诗的理解和赏析，不正确的一项是', 'Choices..."
1,2,隆冬之际，西伯利亚的寒流（笼罩/席卷）欧亚大陆，狂风肆虐，草木凋凌，而那些春天的元素——温暖...,[{'Question': '1. 文中“肆虐”“凋凌”“昼夜兼程”“版图”“稚嫩”“嫣然”...
2,3,材料一：日前，中国科学院在京召开新闻发布会对外宣布“墨子号”进行科学实验卫星提前并圆满实现全...,"[{'Question': '下列对材料相关内容的理解，不正确的一项是', 'Choices..."
3,4,纯礼字彝叟以父仲淹荫知陵台令兼永安县永昭陵建京西转运使配木石砖甓及工徒于一路独永安不受令使者...,[{'Question': '下列对文中“纯礼字彝叟以父仲淹荫知陵台令兼永安县永昭陵建京西转...
4,5,中国传统音乐包括民间音乐、宗教音乐、文人音乐、宫廷音乐等类别，其中文人音乐的代表主要就是古琴...,"[{'Question': '下面填入文中括号内的语句，衔接最恰当的一项是', 'Choic..."


In [5]:
print(train['Content'].iloc[1])
print(train['Questions'].iloc[1])

隆冬之际，西伯利亚的寒流（笼罩/席卷）欧亚大陆，狂风肆虐，草木凋凌，而那些春天的元素——温暖、雨水、绿叶、鲜花，都集结在位于热带的海南岛。海南岛就像是一艘花船，（系/停）在雷州半岛上，满载寒冬大陆的梦幻和想象。每年，从广州向漠河，春天昼夜兼程，都要进行一次生命版图的（扩展/扩充）。他像赤足奔跑的孩子，一路上用稚嫩的声音轻轻呼唤，于是万物苏醒，盛装应和，可谓“东风好作阳和使，      。”迢迢旅途中，气候的巨大差异，导致众多物种中只能有限地参与这一盛会。木棉花花朵硕大，是南国花中豪杰，“一声铜鼓催开，千树珊瑚齐列，”但她终究无法走出岭南。当春天行经长江、黄河流域时，出场的是桃花、杏花等新主角，“桃花嫣然出篱笑，          ”，然而她们却无法追随春天深入雪国，陆续抱憾退出，随后登场的便是白杨、连翘等北国耐寒植物。
[{'Question': '1. 文中“肆虐”“凋凌”“昼夜兼程”“版图”“稚嫩”“嫣然”“抱憾退出”的词语中，有错别字的一项是', 'Choices': ['A. 肆虐  凋凌', 'B. 集结  昼夜兼程', 'C. 版图  稚嫩', 'D. 嫣然  抱憾退出'], 'Answer': 'A', 'Q_id': '000201'}, {'Question': '依次选用文中括号里的词语，最恰当的一项是', 'Choices': ['A. 席卷  系  扩展', 'B. 笼罩  停  扩展', 'C. 席卷  停  扩充', 'D. 笼罩  系  扩充'], 'Answer': 'A', 'Q_id': '000202'}]


In [6]:
swag = pd.read_csv('swag/train.csv')
swag.head()

,Unnamed: 0,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,0,anetv_jkn6uvmqwh4,3416,Members of the procession walk down the street...,Members of the procession walk down the street...,A drum line,gold,passes by walking down the street playing thei...,has heard approaching them.,arrives and they're outside dancing and asleep.,turns the lead singer watches the performance.,0
1,1,anetv_jkn6uvmqwh4,3417,A drum line passes by walking down the street ...,A drum line passes by walking down the street ...,Members of the procession,gen,are playing ping pong and celebrating one left...,wait slowly towards the cadets.,continues to play as well along the crowd alon...,"continue to play marching, interspersed.",3
2,2,anetv_jkn6uvmqwh4,3415,A group of members in green uniforms walks wav...,A group of members in green uniforms walks wav...,Members of the procession,gold,pay the other coaches to cheer as people this ...,walk down the street holding small horn brass ...,is seen in the background.,are talking a couple of people playing a game ...,1
3,3,anetv_jkn6uvmqwh4,3417,A drum line passes by walking down the street ...,A drum line passes by walking down the street ...,Members of the procession,gen,are playing ping pong and celebrating one left...,wait slowly towards the cadets.,makes a square call and ends by jumping down i...,play and go back and forth hitting the drums w...,3
4,4,anetv_Bri_myFFu4A,2408,The person plays a song on the violin. The man,The person plays a song on the violin.,The man,gold,finishes the song and lowers the instrument.,hits the saxophone and demonstrates how to pro...,finishes massage the instrument again and cont...,continues dancing while the man gore the music...,0


In [7]:
label_map = {'A': '0', 'B': '1', 'C': '2', 'D': '3'}

In [8]:
res = []
for idx, row in tqdm(train.iterrows()):
    content = row['Content']
    questions = row['Questions']
    for question in questions:
        q_id = question['Q_id']
        choices = question['Choices']
        answer = question['Answer']
        question = question['Question']
        modified_choices = ["", "", "", ""]
        for choice_idx, choice in enumerate(choices):
            modified_choices[choice_idx] = choice[2:]
        label = label_map[answer]
        ## Hard-code for swag format!
        res.append(("", 
                    "",
                    q_id, 
                    "", 
                    content, 
                    question, 
                    "", 
                    modified_choices[0], 
                    modified_choices[1], 
                    modified_choices[2], 
                    modified_choices[3], 
                    label))

In [9]:
cols = swag.columns.tolist()
df = pd.DataFrame(res, columns=cols)

print(df.shape)
df.head()

(15425, 12)


,Unnamed: 0,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,,,000101,,奉和袭美抱疾杜门见寄次韵 陆龟蒙虽失春城醉上期，下帷裁遍未裁诗。因吟郢岸百亩蕙，欲采商崖三...,下列对这首诗的理解和赏析，不正确的一项是,,作者写作此诗之时，皮日休正患病居家，闭门谢客，与外界不通音讯。,由于友人患病，原有的约会被暂时搁置，作者游春的诗篇也未能写出。,作者虽然身在书斋从事教学，但心中盼望能走进自然，领略美好春光。,尾联使用了关于沈约的典故，可以由此推测皮日休所患的疾病是目疾。,0
1,,,000201,,隆冬之际，西伯利亚的寒流（笼罩/席卷）欧亚大陆，狂风肆虐，草木凋凌，而那些春天的元素——温暖...,1. 文中“肆虐”“凋凌”“昼夜兼程”“版图”“稚嫩”“嫣然”“抱憾退出”的词语中，有错别字...,,肆虐 凋凌,集结 昼夜兼程,版图 稚嫩,嫣然 抱憾退出,0
2,,,000202,,隆冬之际，西伯利亚的寒流（笼罩/席卷）欧亚大陆，狂风肆虐，草木凋凌，而那些春天的元素——温暖...,依次选用文中括号里的词语，最恰当的一项是,,席卷 系 扩展,笼罩 停 扩展,席卷 停 扩充,笼罩 系 扩充,0
3,,,000301,,材料一：日前，中国科学院在京召开新闻发布会对外宣布“墨子号”进行科学实验卫星提前并圆满实现全...,下列对材料相关内容的理解，不正确的一项是,,量子通信把量子物理与信息技术结合起来，利用量子调控技术对信息进行编码、存储、传输和操纵，可以...,潘建伟研究团队在天宫二号空间站上进行太空量子实验，并计划发射“墨子号”后的第二颗卫星，他对未...,中国继美国、英国、日本之后成为第四个拥有散裂中子源设施的国家，有些日本科学家有了危机感，认为...,在基础科学研究领域，比如使用人造卫星开展科学实验，需要耗费巨额资金，欧洲和日本都还在犹豫不决...,3
4,,,000302,,材料一：日前，中国科学院在京召开新闻发布会对外宣布“墨子号”进行科学实验卫星提前并圆满实现全...,下列对材料相关内容的概括和分析，不正确的一项是,,利用“墨子号”科学实验卫星研究量子密钥发布和量子隐形形态的量子通信技术，对国家信息安全和人类...,量子密钥分发是通过量子态的传输，使双方共享无条件安全的量子密钥，对信息进行一次一密的严格加密...,考虑到千百年来人们对于通信安全的追求从未停止，市场潜力巨大，中国和欧洲都投入巨额资金，首要目...,材料二和材料三中，国外媒体对我国量子通信技术研究的相关情况进行了报道，认为中国无论是投资力度...,2


In [10]:
test = pd.read_json('raw_data/validation.json')

print(test.shape)
test.head()

(1000, 5)


,ID,Content,Questions,Type,Diff
0,1,周鼎传①公讳鼎，字实甫，自为童子时即以言动自持，趋立多中礼。早岁善属文，补邑诸生，坚意嗜读，...,"[{'Question': '文中“自是豪强屏息”中“屏”的意思是', 'Choices':...",11,1
1,2,苏轼字子瞻，眉州眉山人。母程氏亲授以书，闻古今成败，辄能语其要。嘉佑二年，试礼部，主司欧阳修...,"[{'Question': '下列对文中""三年权知礼部贡举会大雪苦寒士坐庭中噤未能言轼宽其禁...",11,6
2,3,“大洋一号”是中国第一艘现在化的综合性远洋科学考察船，自1995年以来，这艘船经历了大洋款产...,[{'Question': '文中“这艘船经历了大洋款产资源研究开发专项的多个远洋调查航次和...,0,4
3,4,心源亭记（明）李贤 ①邓1之儒学，在古城巽隅2，规制宏敞，殿后曰“明伦堂”。堂前道中...,"[{'Question': '第①段理解不正确的一项是', 'Choices': ['A、高...",11,6
4,5,谢贞，字元正，陈郡阳夏人，晋太傅安九世孙也。父蔺，正员外郎，兼散骑常侍。贞幼聪敏，有至性。祖...,"[{'Question': '对下列语句中的字的解释，不正确的一项是', 'Choices'...",11,6


In [11]:
res = []
for idx, row in tqdm(test.iterrows()):
    content = row['Content']
    questions = row['Questions']
    for question in questions:
        q_id = question['Q_id']
        choices = question['Choices']
        question = question['Question']
        modified_choices = ["", "", "", ""]
        for choice_idx, choice in enumerate(choices):
            modified_choices[choice_idx] = choice[2:]
        ## Hard-code for swag format!
        res.append(("", 
                    "",
                    q_id, 
                    "", 
                    content, 
                    question, 
                    "", 
                    modified_choices[0], 
                    modified_choices[1], 
                    modified_choices[2], 
                    modified_choices[3],
                    0))

In [12]:
df_test = pd.DataFrame(res, columns=cols)

print(df_test.shape)
df_test.head()

(2444, 12)


,Unnamed: 0,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,,,000101,,周鼎传①公讳鼎，字实甫，自为童子时即以言动自持，趋立多中礼。早岁善属文，补邑诸生，坚意嗜读，...,文中“自是豪强屏息”中“屏”的意思是,,屏住呼吸,收敛行迹,退避迁徙,抑止打压,0
1,,,000102,,周鼎传①公讳鼎，字实甫，自为童子时即以言动自持，趋立多中礼。早岁善属文，补邑诸生，坚意嗜读，...,文中“多意沮而寝”中“寝”的意思是,,睡觉,丑陋,隐蔽,停止,0
2,,,000103,,周鼎传①公讳鼎，字实甫，自为童子时即以言动自持，趋立多中礼。早岁善属文，补邑诸生，坚意嗜读，...,第④段断句正确的一项是,,立捕两奴/及舟械至奴/犹称相国/从者意不下。,立捕两奴/及舟/械至奴/犹称相国/从者意不下。,立捕两奴/及舟械至奴/犹称相国从者/意不下。,立捕两奴及舟械/至奴/犹称相国从者/意不下。,0
3,,,000201,,苏轼字子瞻，眉州眉山人。母程氏亲授以书，闻古今成败，辄能语其要。嘉佑二年，试礼部，主司欧阳修...,"下列对文中""三年权知礼部贡举会大雪苦寒士坐庭中噤未能言轼宽其禁约使得尽技巡铺内侍每摧辱举子且...",,三年/权知礼部贡举/会大雪苦寒/士坐庭中/噤未能言/轼宽其禁约/使得尽技/巡辅内侍每摧辱举子...,三年/权知礼部贡举/会大雪苦寒/士坐庭中噤/未能言轼/宽其禁约/使得尽技/巡辅内侍每摧辱举...,三年/权知礼部贡举/会大雪苦寒/士坐庭中噤/未能言轼/宽其禁约/使得尽技巡辅内侍/每摧辱举...,三年/权知礼部贡举/会大雪苦寒/士坐庭中/噤未能言/轼宽其禁约/使得尽技巡辅内侍/每摧辱举...,0
4,,,000202,,苏轼字子瞻，眉州眉山人。母程氏亲授以书，闻古今成败，辄能语其要。嘉佑二年，试礼部，主司欧阳修...,下列对原文有关内容的概括和分析，不正确的一项是,,苏轼自幼聪颖，深受时贤赏识。母亲亲自为他授课，他往往能说出要点。欧阳修十分看重他，曾对梅圣俞...,苏轼因势利导，利用新法便民。当时王安石创行新法，他上书论其不便；新政下达，他常常设法使这些法...,苏轼直面饥疫，解救受灾百姓。他在任职杭州时遭遇旱灾病疫，减免上供米三分之一纾缓灾情；同时又集...,苏轼天赋异禀，为文得心应手。他从父习文，又极具才华，作文如行云流水，行止有度，嬉笑怒骂之辞，...,0


In [13]:
if DEBUG:
    df.iloc[:50].to_csv('train.csv', index=False)
    df.iloc[-50:].to_csv('valid.csv', index=False)
    df_test.iloc[:50].to_csv('test.csv', index=False)
else:
    df.iloc[:13000].to_csv('train.csv', index=False)
    df.iloc[13000:].to_csv('valid.csv', index=False)
    df_test.to_csv('test.csv', index=False)

In [14]:
# !rm -rf /media/user01/67a19c61-e7f0-4409-ac85-5eee69511615/haihua_roberta

In [15]:
!python ./transformers/examples/multiple-choice/run_swag.py \
--model_name_or_path 'hfl/chinese-roberta-wwm-ext' \
--do_train \
--do_eval \
--do_predict \
--max_seq_length 512 \
--train_file train.csv \
--validation_file valid.csv \
--test_file test.csv \
--learning_rate 2e-5 \
--num_train_epochs 5 \
--output_dir '/media/user01/67a19c61-e7f0-4409-ac85-5eee69511615/haihua_roberta' \
--gradient_accumulation_steps 8 \
--per_device_eval_batch_size 2 \
--per_device_train_batch_size 2 \
--overwrite_output

02/18/2021 09:38:56 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1distributed training: False, 16-bits training: False
02/18/2021 09:38:56 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=/media/user01/67a19c61-e7f0-4409-ac85-5eee69511615/haihua_roberta, overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=True, evaluation_strategy=EvaluationStrategy.NO, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, gradient_accumulation_steps=8, eval_accumulation_steps=None, learning_rate=2e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=5.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_steps=0, logging_dir=runs/Feb18_09-38-56_user01-X9DRG-HF, logging_first_step=False, logging_steps=500, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt_level=O1, fp16_backend=auto, local_rank=-1,

In [16]:
test_results = pd.read_csv('/media/user01/67a19c61-e7f0-4409-ac85-5eee69511615/haihua_roberta/test_results.txt')

print(test_results.shape)
test_results.head()

(2444, 2)


,id,label
0,0,C
1,1,B
2,2,C
3,3,A
4,4,D


In [17]:
sub = pd.read_csv('raw_data/sample.csv')
sub['label'] = test_results['label']

print(sub.shape)
sub.head()

(2444, 2)


,id,label
0,101,C
1,102,B
2,103,C
3,201,A
4,202,D


In [18]:
sub.to_csv('baseline_roberta.csv', index=False)